In [2]:
import warnings
warnings.simplefilter(action='ignore')
import os
import sys
parent_dir = os.path.join(os.getcwd(), '..')
if parent_dir not in sys.path: sys.path.append(parent_dir)
from sklearn.ensemble import RandomForestRegressor
import pandas as pd
from sklearn.metrics import root_mean_squared_log_error
from modules.utility import get_date_details, one_hot, upload
from config import args


In [3]:
OUTPUT_FILE_NAME = "sample_submission_random_forest_random_patches.csv"

In [4]:
df = pd.read_csv(args.train_csv)
test_df = pd.read_csv(args.test_csv)

In [5]:
df['date'] = pd.to_datetime(df['date'])

new_date = get_date_details(df)
new_df, dummies = one_hot(new_date)
combined = pd.concat([new_df, dummies], axis=1)

In [6]:
test_df['date'] = pd.to_datetime(test_df['date'])
test_new_date = get_date_details(test_df)
test_new_df, test_dummies = one_hot(test_new_date)
test_combined = pd.concat([test_new_df, test_dummies], axis=1)

In [7]:

from sklearn.model_selection import train_test_split
x, y = combined.drop('sales', axis=1), combined['sales']
train_x, test_x, train_y, test_y = train_test_split(x.drop('date', axis=1), y, test_size=.2)



In [ ]:

rf = RandomForestRegressor(
    n_estimators=100,      
    max_features='sqrt',    
    bootstrap=True,         
    random_state=42        
)
rf.fit(train_x.values, train_y.values)

In [12]:

root_mean_squared_log_error(test_y, rf.predict(test_x).clip(0))

1.224234734795313

In [14]:
upload(
    pred=rf.predict(test_combined.drop('date', axis=1)).clip(0), 
    test_df=test_df, 
    filename=os.path.join(args.output, OUTPUT_FILE_NAME)
)